# Mobile insights

In [8]:
import json

with open('files/TMRW_mob.json') as file:
    input_dev = json.load(file)
    
input_dev

{'reports': [{'columnHeader': {'dimensions': ['ga:deviceCategory'],
    'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
       'type': 'INTEGER'},
      {'name': 'ga:bounces', 'type': 'INTEGER'},
      {'name': 'ga:goal1Completions', 'type': 'INTEGER'}]}},
   'data': {'rows': [{'dimensions': ['desktop'],
      'metrics': [{'values': ['4263', '2251', '117']}]},
     {'dimensions': ['mobile'],
      'metrics': [{'values': ['2352', '1468', '26']}]},
     {'dimensions': ['tablet'],
      'metrics': [{'values': ['278', '155', '3']}]}]}}]}

In [9]:
# Define dimensions list
input_dev_dimensions = input_dev['reports'][0]['columnHeader']['dimensions']

input_dev_dimensions

['ga:deviceCategory']

In [10]:
# Define metrics list
input_dev_metrics = input_dev['reports'][0]['columnHeader']['metricHeader']['metricHeaderEntries']

def create_metric_list(raw_data):
    lst = []
    for item in raw_data:
        lst.append(item['name'])
    return lst

input_dev_metrics = create_metric_list(input_dev_metrics)

input_dev_metrics

['ga:sessions', 'ga:bounces', 'ga:goal1Completions']

In [11]:
# Create input_dev_data

input_dev_data = input_dev['reports'][0]['data']['rows']

input_dev_data

[{'dimensions': ['desktop'], 'metrics': [{'values': ['4263', '2251', '117']}]},
 {'dimensions': ['mobile'], 'metrics': [{'values': ['2352', '1468', '26']}]},
 {'dimensions': ['tablet'], 'metrics': [{'values': ['278', '155', '3']}]}]

In [12]:
# Define Sessions dict

dev_sessions = {}
dev_bounces = {}
dev_conversions = {}


for device in input_dev_data:
    
    device_name = device['dimensions'][0]
        
    sessions_metric = int(device['metrics'][0]['values'][0])
    bounces_metric = int(device['metrics'][0]['values'][1])
    conv_metric = int(device['metrics'][0]['values'][2])
    
    dev_sessions[device_name] = sessions_metric
    dev_bounces[device_name] = bounces_metric
    dev_conversions[device_name] = conv_metric
    
print(dev_sessions)
print(dev_bounces)
print(dev_conversions)
    
    
new_data = [dev_sessions, dev_bounces, dev_conversions]
new_data

{'desktop': 4263, 'mobile': 2352, 'tablet': 278}
{'desktop': 2251, 'mobile': 1468, 'tablet': 155}
{'desktop': 117, 'mobile': 26, 'tablet': 3}


[{'desktop': 4263, 'mobile': 2352, 'tablet': 278},
 {'desktop': 2251, 'mobile': 1468, 'tablet': 155},
 {'desktop': 117, 'mobile': 26, 'tablet': 3}]

In [13]:
# Create dev_data

dev_data = {}  

for metric_list in new_data:
    for device in metric_list:
        
        dev_data[device] = {'sessions':0,
                           'bounces':0,
                           'conversions':0}        
        
        dev_data[device]['sessions'] = dev_sessions[device]      
        dev_data[device]['bounces'] = dev_bounces[device]
        dev_data[device]['conversions'] = dev_conversions[device]
        
dev_data

{'desktop': {'bounces': 2251, 'conversions': 117, 'sessions': 4263},
 'mobile': {'bounces': 1468, 'conversions': 26, 'sessions': 2352},
 'tablet': {'bounces': 155, 'conversions': 3, 'sessions': 278}}

In [14]:
# Group MOBILE and TABLET into MOBILE DEVICES
dev_data['mobile devices'] = {}

for device in dev_data:
    
    if device == 'mobile' or device == 'tablet':
        metrics = dev_data[device]
        
        for metric in metrics:
            dev_data['mobile devices'][metric] = 0
            dev_data['mobile devices'][metric] += dev_data['mobile'][metric]+dev_data['tablet'][metric]
            
del dev_data['mobile']
del dev_data['tablet']
            
dev_data

{'desktop': {'bounces': 2251, 'conversions': 117, 'sessions': 4263},
 'mobile devices': {'bounces': 1623, 'conversions': 29, 'sessions': 2630}}

In [15]:
# Calculate CONVERSION RATE

for device in dev_data:
    dev_data[device]['conversion rate'] = 0
    dev_data[device]['conversion rate'] = dev_data[device]['conversions']/dev_data[device]['sessions']
    
dev_data

{'desktop': {'bounces': 2251,
  'conversion rate': 0.02744546094299789,
  'conversions': 117,
  'sessions': 4263},
 'mobile devices': {'bounces': 1623,
  'conversion rate': 0.011026615969581748,
  'conversions': 29,
  'sessions': 2630}}

### Print

In [16]:
# Calculate how conversions can increase in the next 4 monthes

conv_increase =  int(dev_data['mobile devices']['sessions'] * dev_data['desktop']['conversion rate'] - dev_data['mobile devices']['conversions'])

print ("Have %s more conversions per month by optmising mobile UX" % conv_increase)

Have 43 more conversions per month by optmising mobile UX


In [17]:
mobile_CR = dev_data['mobile devices']['conversion rate']

print ('Only %s ' % ("%.2f" % (mobile_CR*100) +'% of all mobile visits end up completing a conversion' ))

Only 1.10% of all mobile visits end up completing a conversion 


### Visualisation

In [29]:
import plotly

In [67]:
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly import graph_objs as go
from __future__ import division


py.sign_in('m-nudha', 'NlSXxWgqIy6tgiVQIi6e')

In [68]:
mob_metrics = []

for metric in dev_data['mobile devices']:
    if metric == "sessions" or metric == "bounces" or metric == "conversions":
        mob_metrics.append(dev_data['mobile devices'][metric])

mob_metrics

[2630, 1623, 29]

In [69]:
# chart stages data
values = mob_metrics
phases = ['All users', 'Engaged users', 'Converted users']

# color of each funnel section
colors = ['rgb(32,155,160)', 'rgb(253,93,124)', 'rgb(182,231,235)']

In [70]:
n_phase = len(phases)
plot_width = 400

# height of a section and difference between sections 
section_h = 100
section_d = 10

# multiplication factor to calculate the width of other sections
unit_width = plot_width / max(values)

# width of each funnel section relative to the plot width
phase_w = [int(value * unit_width) for value in values]

# plot height based on the number of sections and the gap in between them
height = section_h * n_phase + section_d * (n_phase - 1)

In [71]:
# list containing all the plot shapes
shapes = []

# list containing the Y-axis location for each section's name and value text
label_y = []

for i in range(n_phase):
        if (i == n_phase-1):
                points = [phase_w[i] / 2, height, phase_w[i] / 2, height - section_h]
        else:
                points = [phase_w[i] / 2, height, phase_w[i+1] / 2, height - section_h]

        path = 'M {0} {1} L {2} {3} L -{2} {3} L -{0} {1} Z'.format(*points)

        shape = {
                'type': 'path',
                'path': path,
                'fillcolor': colors[i],
                'line': {
                    'width': 1,
                    'color': colors[i]
                }
        }
        shapes.append(shape)
        
        # Y-axis location for this section's details (text)
        label_y.append(height - (section_h / 2))

        height = height - (section_h + section_d)

In [74]:
# For phase names
label_trace = go.Scatter(
    x=[-350]*n_phase,
    y=label_y,
    mode='text',
    text=phases,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15
    )
)
 
# For phase values
value_trace = go.Scatter(
    x=[350]*n_phase,
    y=label_y,
    mode='text',
    text=values,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15
    )
)

data = [label_trace, value_trace]
 
layout = go.Layout(
    title="<b>Funnel Chart</b>",
    titlefont=dict(
        size=20,
        color='rgb(203,203,203)'
    ),
    shapes=shapes,
    height=560,
    width=800,
    showlegend=False,
    paper_bgcolor='rgba(44,58,71,1)',
    plot_bgcolor='rgba(44,58,71,1)',
    xaxis=dict(
        showticklabels=False,
        zeroline=False,
    ),
    yaxis=dict(
        showticklabels=False,
        zeroline=False
    )
)
 
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)